In [1]:
import pandas as pd
import numpy as np
import urllib3
from bs4 import BeautifulSoup

In [4]:
file='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
http = urllib3.PoolManager()
r=http.request('GET',file)
soup=BeautifulSoup(r.data,'lxml')

table=soup.find('table')

we have now the data of table which we need

In [5]:
Col=table.findAll('th')
key=[]
for col in Col:
    key.append(col.string.replace('\n',''))  
key

['Postal code', 'Borough', 'Neighborhood']

these are the title of our columns, we will extract their contents

In [6]:
rows = table.findAll('tr')
key0 = []
key1 = []
key2 = []
for row in rows:
    
    cells=row.findAll('td')
    if len(cells)>0:
        key0.append(cells[0].string.replace('\n',''))
        key1.append(cells[1].string.replace('\n',''))
        key2.append(cells[2].string.replace('\n',''))

and build a dictionary with these lists

In [9]:
dico={key[0]:key0,key[1]:key1,key[2]:key2}
df=pd.DataFrame(dico)

In [10]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [12]:
df.dtypes

Postal code     object
Borough         object
Neighborhood    object
dtype: object

we will first remove the unassignet boroughs

In [13]:
dfD_index=df[df['Borough']=='Not assigned'].index
df.drop(dfD_index,axis=0,inplace=True)

In [14]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


we reset the index and create a new one:

In [15]:
df=df.reset_index()
df.drop(columns=['index'],inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


we search if we have assigned boroughs with unassigned neighborhood :

In [16]:
dfNnull=df['Neighborhood'].isna().value_counts()
dfNnull

False    103
Name: Neighborhood, dtype: int64

we have 0 assigned boroughs with unassigned neighborhood

we remplace the "/" with a coma in the content of Neighborhood like for example in Regent Park / Harbourfront

In [18]:
df=df.replace("/",",",regex=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [19]:
df.shape

(103, 3)